# Variational Quantum Circuit Compiling

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## Overview

Variational quantum circuit compilation is the process of simulating an unknown unitary operator by optimizing a parameterized quantum circuit. In this tutorial we will discuss two cases of unknown unitary operators. One is that the target $U$ is given as a matrix form, the other is that the $U$ is given as a black-box. We show how to obtain the loss function in both cases in Paddle Quantum. With auto-differentiation and optimizer provided with PaddlePaddle, we could easily approximate $U$ into a trainable sequence of quantum gates (here we use $V(\vec{\theta})$ to denote the unitary operator represented by the sequence of parameterized quantum gates, and for simplicity, we use $V$ below). Finally, we validate the optimized circuit by comparing the trace distance of various output density matrices transformed by the approximate circuit and the target $U$.

## Background

Earlier compilations of classical computers transformed binary numbers into electrical signals to drive the computer's electronic devices to perform operations, and then gradually developed into an assembly language for easy processing and writing. For quantum computers, similar to classical compilation, quantum compilation is a process of converting the unitary in a quantum algorithm into a series of the quantum gates to implement the algorithm. The current noisy intermediate-scale quantum (NISQ) devices have limitations such as the number of qubits, circuit depth, etc., which pose a great challenge to quantum compilation algorithms. In [1], a quantum compilation algorithm, the Quantum-assisted Quantum Compiling (QAQC), has been proposed for efficient implementation on NISQ devices. The idea of QAQC is to compile the unknown target unitary operator $U$ into the unitary $V$, define the loss function using the gate fidelity, and continuously optimize a variational quantum circuit by minimizing the loss function. But how to measure the similarity of the two unitary operators? Here we consider the probability that the unitary evolution of the $V$ can simulate the $U$, i.e., the degree of overlap between $U|\psi\rangle$ and $V|\psi\rangle$ for the input state $|\psi\rangle$, which is the average of the fidelity on the Haar distribution:

$$
F(U,V)=\int_{\psi}|\langle\psi|V^{\dagger}U|\psi\rangle|^2d\psi,
\tag{1}
$$

When $F(U,V)=1$, there is a $\phi$ such that $V=e^{i\phi}U$, i.e., the two unitary operators differ by a global phase factor, at which point we call $V$ an exact compilation of $U$. When $F(U,V)\geq 1-\epsilon$, we call $V$ an approximate compilation of $U$, where $\epsilon$ is an error and $\epsilon\in[0,1]$. Based on this, we can construct the following loss function:

$$
\begin{aligned} C(U,V)&=\frac{d+1}{d}(1-F(U,V))\\
&=1-\frac{1}{d^2}|\langle V,U\rangle|^2\\
&=1-\frac{1}{d^2}|\text{tr}(V^{\dagger} U)|^2,
\end{aligned}
\tag{2}
$$

where $n$ is the number of qubits, $d=2^n$ and $\frac{1}{d^2}|\text{tr}(V^{\dagger} U)|^2$ is the gate fidelity.

From (2), we have that $C(V,U)=0$ if and only if $F(U,V)=1$, so we can obtain $V$ that approximates the target unitary operator $U$ by training a sequence of rotational gates with adjustable angles to minimize the loss function.

## The First Scenario - Matrix Form of $U$

In the first case, we suppose that $U$ is given in the form of a matrix. Taking the Toffoli gate as an example, we note its matrix representation as $U_0$. We wish to construct a quantum neural network (QNN, i.e., parameterized quantum circuit) to obtain an approximate circuit decomposition of $U_0$ by training.

Let us import the necessary packages:

In [1]:
import numpy as np
import paddle
import paddle_quantum
from paddle_quantum.ansatz import Circuit
from paddle_quantum.linalg import dagger
from paddle_quantum.state import random_state, zero_state

/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/openfermion/hamiltonians/hartree_fock.py:11: DeprecationWarning: Please use `OptimizeResult` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import OptimizeResult
/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in 

We need to get the Toffoli gate's unitary matrix:

In [2]:
n = 3  # Number of qubits

# The matrix form of Toffoli gate
U_0 = paddle.to_tensor(np.matrix([[1, 0, 0, 0, 0, 0, 0, 0],
                                  [0, 1, 0, 0, 0, 0, 0, 0],
                                  [0, 0, 1, 0, 0, 0, 0, 0],
                                  [0, 0, 0, 1, 0, 0, 0, 0],
                                  [0, 0, 0, 0, 1, 0, 0, 0],
                                  [0, 0, 0, 0, 0, 1, 0, 0],
                                  [0, 0, 0, 0, 0, 0, 0, 1],
                                  [0, 0, 0, 0, 0, 0, 1, 0]],
                       dtype="float32"))

### Constructing quantum circuits

Different QNNs have different expressibility. Here we choose the `complex_entangled_layer()` function built-in in Paddle Quantum to construct QNN:

In [3]:
# Constructing quantum circuit
def qcircuit(n, D):
    # Initialize the circuit
    cir = Circuit(n)
    # Call the built-in QNN template
    cir.complex_entangled_layer('full', D)

    return cir


### Setting up the training model - loss function

Next we define the loss function $C(U,V) = 1-\frac{1}{d^2}|\text{tr}(V^{\dagger} U)|^2$ and training parameters in order to optimize the parameterized circuit. 

In [4]:
# Define the loss function
def loss_func(cir, target_u):
    # The matrix form of the circuit
    V = cir.unitary_matrix()
    # Construct Eq.(1) as the loss function
    loss = 1 - (dagger(V).matmul(target_u).trace().abs() / V.shape[0]) ** 2

    return loss 

### Setting up the training model - model parameters

Before training the QNN, we also need to set some training hyperparameters, mainly the depth (D) of repeated blocks, the learning rate (LR), and the number of iterations (ITR). Here we set the learning rate to 0.2 and the number of iterations to 150. The reader can adjust the hyperparameters to observe the impact on the training effect.

In [5]:
D = 5  # Set the depth of QNN
LR = 0.2  # Set the learning rate
ITR = 150   # Set the number of optimization iterations

### Training

In [6]:
cir = qcircuit(n, D)
# Using Adam optimizer to obtain relatively good convergence
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=cir.parameters())

# Optimization loop
for itr in range(1, ITR + 1):
    loss = loss_func(cir, U_0)
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    if itr % 30 == 0:
        print("iter:", itr, "loss:", "%.4f" % loss.numpy())
    if itr == ITR:
        print("\nThe trained circuit:")
        print(cir)

theta_opt = cir.parameters()
print("The trained parameter theta:\n", np.around(theta_opt, decimals=3))

/home/zl/miniconda3/envs/pq/lib/python3.8/site-packages/paddle/fluid/framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


iter: 30 loss: 0.1692
iter: 60 loss: 0.1476
iter: 90 loss: 0.1465
iter: 120 loss: 0.1464
iter: 150 loss: 0.1464

The trained circuit:
--U----*---------x----U----*---------x----U----*---------x----U----*---------x----U----*---------x--
       |         |         |         |         |         |         |         |         |         |  
--U----x----*----|----U----x----*----|----U----x----*----|----U----x----*----|----U----x----*----|--
            |    |              |    |              |    |              |    |              |    |  
--U---------x----*----U---------x----*----U---------x----*----U---------x----*----U---------x----*--
                                                                                                    
The trained parameter theta:
 [[[[ 3.142e+00  2.169e+00  1.385e+00]
   [-1.571e+00  1.569e+00  2.356e+00]
   [ 3.141e+00  5.280e+00 -1.279e+00]]]


 [[[ 4.712e+00  1.570e+00  3.141e+00]
   [ 3.927e+00  3.138e+00  3.139e+00]
   [ 4.713e+00  4.712e+00  6.283e+00

In this case, we construct a five-layer QNN and train it with an Adam optimizer. After around 150 iterations, the loss function reaches 0.

### Validation of results

In the following, we randomly select 10 density matrices, which are evolved by the target unitary operator $U$ and the approximate unitary operator $V$. Then we calculate the trace distance $ d(\rho, \sigma) = \frac{1}{2}\text{tr}\sqrt{(\rho-\sigma)^{\dagger}(\rho-\sigma)}$ between the real output `real_output` $\rho$ and the approximate output `simulated_output` $\sigma$. The smaller the trace distance, the better the approximation effect.

In [7]:
s = 10 # Set the number of randomly generated density matrices

for i in range(s):
    paddle_quantum.set_backend('density_matrix')  # change to density matrix mode     
    sampled = random_state(3) # randomly generated density matrix of 3 qubits sampled
    simulated_output = paddle.matmul(paddle.matmul(cir.unitary_matrix(), sampled.data), dagger(cir.unitary_matrix())) # sampled after approximate unitary evolution
    real_output = paddle.matmul(paddle.matmul(paddle.to_tensor(U_0), sampled.data), dagger(paddle.to_tensor(U_0))) # sampled after target unitary evolution
    print('sample', i + 1, ':')
    A = simulated_output.numpy() - real_output.numpy()
    d = 1 / 2 * np.sum(np.abs(np.linalg.eigvals(A)))
    print('  trace distance is', np.around(d, decimals=5)) # print trace distance


sample 1 :
  trace distance is 0.19792
sample 2 :
  trace distance is 0.2463
sample 3 :
  trace distance is 0.2343
sample 4 :
  trace distance is 0.18177
sample 5 :
  trace distance is 0.18689
sample 6 :
  trace distance is 0.18345
sample 7 :
  trace distance is 0.22967
sample 8 :
  trace distance is 0.22579
sample 9 :
  trace distance is 0.21692
sample 10 :
  trace distance is 0.25945


We can see that the trace distance of each sample after the evolution of $U$ and $V$ is close to 0, which means the $V$ approximates $U$ very well.

## The Second Scenario - Circuit Form of $U$

In the second case, we suppose the $U$ needs approximation is given in the form of a black-box, and we only have access to its input and output. As a results, the fidelity can no longer be computed directly. Instead, it needs to be evaluate by a circuit.
Next we will show how to calculate fidelity with a quantum circuit.

### Calculate fidelity with a quantum circuit

The QNN of QAQC that needs in a large quantum circuit. The whole circuit is shown below, where $U$ denotes the unitary operator to be approximated, and $V^{\dagger}$ is the QNN we want to train. Here we use the Toffoli gate as the black-box.

![circuit](./figures/vqcc-fig-circuit.png "Figure 1: The circuit of the QAQC [1].")
<center>Figure 1: The circuit of the QAQC [1].</center>

The circuit requires a total of $2n$ qubits, and we call the first $n$ qubits system $A$ and the last $n$ qubits system $B$. The whole circuit involves the following three steps:

- First creating a maximally entangled state between $A$ and $B$ by performing Hadamard and CNOT gates.
- Then acting with $U$ on system $A$ and with $V^{\dagger}$ on system $B$ ($V^{\dagger}$ is the complex conjugate of $V$), note that these two gates are performed in parallel.
- Finally measuring in the bell basis(i.e., undoing the CNOTS and Hadamards then measuring in the standard basis).

After the above operation, the probability of the full zero state obtained by the measurement is $\frac{1}{d^2}|\text{tr}(V^{\dagger} U)|^2$. For a detailed explanation of Figure 1 please refer to the literature [1].

Here we use the same QNN that we used in the first case and use the Toffoli gate as the black-bx. 

Next we will implement variational quantum circuit compiling in Paddle Quantum as follows:

In [8]:
n = 3 # Number of qubits

# Construct the total quantum circuit
def qcircuit(n, D):
    
    # Initialize the circuit of 2n qubits 
    cir = Circuit(2 * n)
    cir.h(list(range(n)))
    for i in range(n):
        cir.cnot([i, n + i])

    # Construct the circuit of U
    cir.toffoli([0, 1, 2])

    # Construct QNN
    cir.complex_entangled_layer([3, 4, 5], D)
    
    for l in range(n):
        cir.cnot([n - 1 - l, 2 * n - 1 - l])
    cir.h(list(range(n)))
 
    return cir

### Setting up the training model - loss function

Next we define the loss function $C(U,V) = 1-\frac{1}{d^2}|\text{tr}(V^{\dagger} U)|^2$ and training parameters in order to optimize the QNN. 

In [9]:
# Define loss function
def loss_func(self):
    paddle_quantum.set_backend('density_matrix')      
    # Output the density matrix rho of the quantum state after the circuit
    init_state = zero_state(2 * n)
    rho = cir(init_state)
    # Define loss function
    loss = 1 - paddle.real(rho.data[0][0])

    return loss

### Setting up the training model - model parameters

Here we set the learning rate to 0.1 and the number of iterations to 120. The reader can also adjust them to observe the impact on the training effect.

In [10]:
D = 5  # Set the depth of QNN
LR = 0.1  # Set the learning rate
ITR = 120   # Set the number of optimization iterations

### Training

Then we commence the training process with an Adam optimizer.

In [11]:
cir = qcircuit(n, D)
# Use Adam optimizer for better performance
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=cir.parameters())

# Optimization loop
for itr in range(1, ITR + 1):
    loss = loss_func(cir)
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    # Print training results
    if itr % 20 == 0:
        print("iter:",itr,"loss:","%.4f" % loss.numpy())
    if itr == ITR:
        print("\nThe trained circuit:")
        print('Approximate circuit of U with circuit form input:\n', cir)


iter: 20 loss: 0.4525
iter: 40 loss: 0.1755
iter: 60 loss: 0.0066
iter: 80 loss: 0.0021
iter: 100 loss: 0.0005
iter: 120 loss: 0.0001

The trained circuit:
Approximate circuit of U with circuit form input:
 --H----*--------------*-------------------------------------------------------------------------------------------------------------*----H--
       |              |                                                                                                             |       
--H----|----*---------*--------------------------------------------------------------------------------------------------------*----|----H--
       |    |         |                                                                                                        |    |       
--H----|----|----*----X---------------------------------------------------------------------------------------------------*----|----|----H--
       |    |    |                                                                      

In [12]:
# Storage optimized parameters
theta_opt = cir.parameters()

In this case, we construct a one-layer QNN and train it with a Adam optimizer. After around 100 iterations, the loss function reaches 0.

### Validation of results

Similar to before, we also randomly select 10 density matrices, which are evolved by the target unitary operator $U$ and the approximate unitary operator $V$. Then calculate the trace distance between the real output `real_output` and the approximate output `simulated_output`. The smaller the trace distance, the better the approximation.

In [13]:
s = 10 # Set the number of randomly generated density matrices
for i in range(s):
    paddle_quantum.set_backend('density_matrix')  # change to density matrix mode    
    sampled = random_state(3) # randomly generated density matrix of 4 qubits sampled

    # Construct the circuit of target unitary
    cir_1 = Circuit(3)
    cir.toffoli([0, 1, 2])
    # sampled after target unitary evolution
    real_output = paddle.matmul(paddle.matmul(cir_1.unitary_matrix(), sampled.data), dagger(cir_1.unitary_matrix()))

    # Construct the circuit of approximate unitary
    cir_2 = Circuit(3)
    for j in range(D):
        cir_2.u3(qubits_idx='full', param=theta_opt[j])
        cir_2.cnot(qubits_idx='cycle')
    # sampled after approximate unitary evolution
    simulated_output = paddle.matmul(paddle.matmul(cir_2.unitary_matrix(), sampled.data), dagger(cir_2.unitary_matrix()))

    A = simulated_output.numpy() - real_output.numpy()
    d = 1 / 2 * np.sum(np.abs(np.linalg.eigvals(A)))
    print('sample', i + 1, ':')
    print('  trace distance is',  np.around(d, decimals=5)) # print trace distance

sample 1 :
  trace distance is 0.20575
sample 2 :
  trace distance is 0.2397
sample 3 :
  trace distance is 0.23174
sample 4 :
  trace distance is 0.15452
sample 5 :
  trace distance is 0.2846
sample 6 :
  trace distance is 0.2477
sample 7 :
  trace distance is 0.21579
sample 8 :
  trace distance is 0.1746
sample 9 :
  trace distance is 0.18502
sample 10 :
  trace distance is 0.32651


We can see that the trace distance of each sample after the evolution of $U$ and $V$ is close to 0, which means the $V$ approximates $U$ very well.

## Conclusion

In this tutorial, the variational quantum circuit compiling is carried out from the input form of the target unitary operator as a matrix and as a circuit. The results of the quantum compilation are demonstrated by two simple examples using Paddle Quantum. Then the approximate effect is checked by the trace distance of the quantum states after the evolution of the target unitary and the approximate unitary respectively. Finally the results in Paddle Quantum show that the quantum compilation is good.

_______

## References

[1] Khatri, Sumeet, et al. "Quantum-assisted quantum compiling." [Quantum 3 (2019): 140](https://quantum-journal.org/papers/q-2019-05-13-140/).